In [1]:
import pandas as pd
import numpy as np
import math as m
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re
import os
import get_full_info
from sklearn import linear_model as lm, metrics, ensemble as ens
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from xgboost import XGBClassifier, XGBRegressor
import warnings

2024-04-01 19:30:44.170766: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = get_full_info.df
df['average_annual_rent'] = df['average_annual_rent'].astype('float')
df['average_home_value'] = df['average_home_value'].astype('float')
df['average_income'] = df['average_income'].astype('float')
df.to_csv("full_info.csv", index = False)
df['state'] = df['state'].apply(lambda x: re.sub('District Of Columbia', 'District of Columbia', x))
df = df[df['total_population'] >= 50000]

df_latest = get_full_info.df_realtime
df_latest.to_csv("latest_years.csv", index = False)

In [3]:
df = pd.get_dummies(df, prefix = 'state', columns = ['state'])

for col in df.columns:
    if 'state' in col:
        df[col] = df[col].apply(lambda x: 1 if x == True else 0)
        


In [4]:
std_scl = StandardScaler()

EVERYTHING I CAN FIND ON TSS SAYS THAT IT DOESN'T ACTUALLY SPLIT ON A TIME VARIABLE; RATHER, IT ASSUMES THE DATA IS EVENLY DISTRIBUTED ACROSS AND ORDERED BY TIME, SO THE # OF SPLITS JUST GOES BY INDEX AND EVENLY SPLITS. UNFORTUNATELY, WE HAVE UNEVEN NUMBERS OF OBSERVATIONS, BUT THE MAJORITY HAVE 11 YEARS (THE NUMBER OF YEARS FOR WHICH WE PULLED DATA). SO, I GROUP BY PLACE AND DROP ALL PLACES WITHOUT THE FULL 11 YEARS SO THAT WE CAN SPLIT EVENLY AND SAFELY TRAIN WITHOUT LEAKAGE.

In [5]:
df_count = df.groupby('place').count().reset_index(inplace = False)
places_to_use = list(set(df_count[df_count['year'] == 11]['place'].values.tolist()))

In [6]:
len(places_to_use) #DOUBLECHECK LATER ON THAT EACH SPLIT IS THIS LONG

265

In [7]:
df = df[df['place'].isin(places_to_use)]

In [8]:
tscv = TimeSeriesSplit(n_splits = 10)

In [9]:
df.set_index('year', inplace=True)
df.sort_index(inplace=True)

In [11]:
ex = df.drop(columns = ['place', 'roi'])
why = df['roi']

In [12]:
for i, (train_index, test_index) in enumerate(tscv.split(ex)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

Fold 0:
  Train: index=[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240

In [13]:
# ^^THE SPLIT/PLACE-PRUNING WORKED!

RANDOM FOREST WITH PCA

In [14]:
warnings.filterwarnings("ignore")

# 10X ITERATED STRATIFIED 3-FOLD CROSS-VALIDATED ACCURACY AND PRECISION
steps = [('pca', PCA()), ('rf', ens.RandomForestRegressor(random_state=123))]
model_rf = Pipeline(steps = steps)

# Creating space for different hyperparameters
space = dict()
space['pca__n_components'] = [i for i in range(2, 21)]
space['rf__n_estimators'] = [50, 100, 150, 200, 300, 400, 500]
space['rf__max_depth'] = [None, 5, 10, 14, 19, 25, 30]
space['rf__criterion'] = ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']

# Search
search = RandomizedSearchCV(model_rf, space, scoring='neg_root_mean_squared_error', cv=tscv, n_jobs=-1, verbose = 2)
result = search.fit(ex, why.values.ravel())

#Print Result
print('Best Score: %s' % result.best_score_)
rf_pca_score = result.best_score_
print('Best Hyperparameters: %s' % result.best_params_)
rf_pca_params = result.best_params_
%store rf_pca_params

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Best Score: -0.09873278252277962
Best Hyperparameters: {'rf__n_estimators': 200, 'rf__max_depth': 19, 'rf__criterion': 'friedman_mse', 'pca__n_components': 18}
Stored 'rf_pca_params' (dict)


RANDOM FOREST WITHOUT PCA

In [15]:
# 10X ITERATED STRATIFIED 3-FOLD CROSS-VALIDATED ACCURACY AND PRECISION
model_rf_withoutPCA = ens.RandomForestRegressor(random_state=123)

# Creating space for different hyperparameters
space = dict()
space['n_estimators'] = [50, 100, 150, 200, 300, 400, 500]
space['max_depth'] = [None, 5, 10, 14, 19, 25, 30]
space['criterion'] = ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']

# Search
search = RandomizedSearchCV(model_rf_withoutPCA, space, scoring='neg_root_mean_squared_error', cv=tscv, n_jobs=-1, verbose = 2)
result = search.fit(ex, why.values.ravel())

#Print Result
print('Best Score: %s' % result.best_score_)
rf_no_pca_score = result.best_score_
print('Best Hyperparameters: %s' % result.best_params_)
rf_no_pca_params = result.best_params_
%store rf_no_pca_params

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Best Score: -0.09252717157516648
Best Hyperparameters: {'n_estimators': 500, 'max_depth': 19, 'criterion': 'squared_error'}
Stored 'rf_no_pca_params' (dict)
[CV] END pca__n_components=3, rf__criterion=absolute_error, rf__max_depth=25, rf__n_estimators=500; total time=  27.2s
[CV] END pca__n_components=20, rf__criterion=absolute_error, rf__max_depth=None, rf__n_estimators=300; total time= 2.3min
[CV] END criterion=squared_error, max_depth=5, n_estimators=500; total time=  19.3s
[CV] END criterion=friedman_mse, max_depth=None, n_estimators=200; total time=  47.6s
[CV] END criterion=squared_error, max_depth=19, n_estimators=500; total time= 2.1min
[CV] END criterion=absolute_error, max_depth=5, n_estimators=50; total time=  20.7s
[CV] END pca__n_components=3, rf__criterion=absolute_error, rf__max_depth=25, rf__n_estimators=500; total time=   5.1s
[CV] END pca__n_components=18, rf__criterion=friedman_mse, rf__max_depth=19, rf__

In [16]:
if rf_no_pca_score < rf_pca_score: #IN CASE OF TIE GO NO PCA SINCE IT'S COMPUTATIONALLY CHEAPER
    print("with pca")
    rf_params = rf_pca_params
else:
    print("no pca")
    rf_params = rf_no_pca_params
    
# STORE FOR OTHER NOTEBOOKS
%store rf_params

no pca
Stored 'rf_params' (dict)
[CV] END pca__n_components=3, rf__criterion=absolute_error, rf__max_depth=25, rf__n_estimators=500; total time=   2.4s
[CV] END pca__n_components=18, rf__criterion=friedman_mse, rf__max_depth=19, rf__n_estimators=200; total time=   4.5s
[CV] END pca__n_components=18, rf__criterion=friedman_mse, rf__max_depth=19, rf__n_estimators=200; total time=  11.0s
[CV] END pca__n_components=20, rf__criterion=absolute_error, rf__max_depth=None, rf__n_estimators=300; total time=  48.8s
[CV] END pca__n_components=9, rf__criterion=absolute_error, rf__max_depth=None, rf__n_estimators=100; total time=  35.6s
[CV] END criterion=squared_error, max_depth=5, n_estimators=500; total time=  30.0s
[CV] END criterion=squared_error, max_depth=25, n_estimators=150; total time=   9.6s
[CV] END criterion=squared_error, max_depth=25, n_estimators=150; total time=  31.1s
[CV] END criterion=friedman_mse, max_depth=None, n_estimators=50; total time=   0.8s
[CV] END criterion=friedman_ms

ADABOOST W/PCA

In [17]:
warnings.filterwarnings("ignore")

# 10X ITERATED STRATIFIED 3-FOLD CROSS-VALIDATED ACCURACY AND PRECISION
steps = [('pca', PCA()), ('ada', AdaBoostRegressor(random_state=123))]
model_ada = Pipeline(steps = steps)

# Creating space for different hyperparameters
space = dict()
space['pca__n_components'] = [i for i in range(2, 21)]
space['ada__n_estimators'] = [10, 50, 100, 150, 200, 250, 300]
space['ada__learning_rate'] = [10**i for i in range(-7,1)]
# space['ada__algorithm'] = ['SAMME', 'SAMME.R']

# Search
search = RandomizedSearchCV(model_ada, space, scoring='neg_root_mean_squared_error', cv=tscv, n_jobs=-1, verbose = 2)
result = search.fit(ex, why.values.ravel())

#Print Result
print('Best Score: %s' % result.best_score_)
ada_pca_score = result.best_score_
print('Best Hyperparameters: %s' % result.best_params_)
ada_pca_params = result.best_params_
%store ada_pca_params

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Best Score: -0.12576087662604366
Best Hyperparameters: {'pca__n_components': 18, 'ada__n_estimators': 300, 'ada__learning_rate': 0.1}
Stored 'ada_pca_params' (dict)


ADABOOST WITHOUT PCA

In [18]:
#Ada Boost
# 10X ITERATED STRATIFIED 3-FOLD CROSS-VALIDATED ACCURACY AND PRECISION
model_ada_withoutPCA = AdaBoostRegressor(random_state=123)

# Creating space for different hyperparameters
space = dict()
space['n_estimators'] = [10, 50, 100, 150, 200, 250, 300]
space['learning_rate'] = [10**i for i in range(-7,1)]
# space['algorithm'] = ['SAMME', 'SAMME.R']

# Search
search = RandomizedSearchCV(model_ada_withoutPCA, space, scoring='neg_root_mean_squared_error', cv=tscv, n_jobs=-1, verbose = 2)
result = search.fit(ex, why.values.ravel())

#Print Result
print('Best Score: %s' % result.best_score_)
ada_no_pca_score = result.best_score_
print('Best Hyperparameters: %s' % result.best_params_)
ada_no_pca_params = result.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Best Score: -0.10245864232174735
Best Hyperparameters: {'n_estimators': 300, 'learning_rate': 1}


In [19]:
if ada_no_pca_score < ada_pca_score: #IN CASE OF TIE GO NO PCA SINCE IT'S COMPUTATIONALLY CHEAPER
    print("with pca")
    ada_params = ada_pca_params
else:
    print("no pca")
    ada_params = ada_no_pca_params
    
# STORE FOR OTHER NOTEBOOKS
%store ada_params

no pca
Stored 'ada_params' (dict)


XGBOOST WITH PCA

In [20]:
#XG Boost
from xgboost import XGBRegressor
warnings.filterwarnings("ignore")

# 10X ITERATED STRATIFIED 3-FOLD CROSS-VALIDATED ACCURACY AND PRECISION
steps = [('pca', PCA()), ('xg', XGBRegressor(eval_metric = "error", use_label_encoder =False, random_state=123))]
model_xg = Pipeline(steps = steps)

# Creating space for different hyperparameters
space = dict()
space['pca__n_components'] = [12, 13, 14, 15, 16]
space['xg__eta'] = [a for a in np.arange(0.1, 1.1, 0.1)]
space['xg__max_depth'] = [4, 5, 6, 7]
space['xg__scale_pos_weight'] = [0.25, 1]

# Search
search = RandomizedSearchCV(model_xg, space, scoring='neg_root_mean_squared_error', cv=tscv, n_jobs=-1, verbose = 2)
result = search.fit(ex, why.values.ravel())

#Print Result
print('Best Score: %s' % result.best_score_)
xgboost_pca_score = result.best_score_
print('Best Hyperparameters: %s' % result.best_params_)
xgboost_pca_params = result.best_params_
%store xgboost_pca_params

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Best Score: -0.09237308216267451
Best Hyperparameters: {'xg__scale_pos_weight': 0.25, 'xg__max_depth': 7, 'xg__eta': 0.30000000000000004, 'pca__n_components': 13}
Stored 'xgboost_pca_params' (dict)


XGBOOST WITHOUT PCA

In [21]:
#XG Boost
warnings.filterwarnings("ignore")

# 10X ITERATED STRATIFIED 3-FOLD CROSS-VALIDATED ACCURACY AND PRECISION
model_xg_withoutPCA = XGBRegressor(eval_metric = "error", use_label_encoder =False, random_state=123)

# Creating space for different hyperparameters
space = dict()
space['eta'] = [a for a in np.arange(0.1, 1.1, 0.1)]
space['max_depth'] = [4, 5, 6, 7]
space['scale_pos_weight'] = [0.25, 1]

# Search
search = RandomizedSearchCV(model_xg_withoutPCA, space, scoring='neg_root_mean_squared_error', cv=tscv, n_jobs=-1, verbose = 2)
result = search.fit(ex, why.values.ravel())

#Print Result
print('Best Score: %s' % result.best_score_)
xgboost_no_pca_score = result.best_score_
print('Best Hyperparameters: %s' % result.best_params_)
xgboost_no_pca_params = result.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Best Score: -0.08657203255310036
Best Hyperparameters: {'scale_pos_weight': 0.25, 'max_depth': 6, 'eta': 0.1}


In [22]:
if xgboost_no_pca_score < xgboost_pca_score: #IN CASE OF TIE GO NO PCA SINCE IT'S COMPUTATIONALLY CHEAPER
    print("with pca")
    xgboost_params = xgboost_pca_params
else:
    print("no pca")
    xgboost_params = xgboost_no_pca_params
    
# STORE FOR OTHER NOTEBOOKS
%store xgboost_params

no pca
Stored 'xgboost_params' (dict)
[CV] END ada__learning_rate=1e-05, ada__n_estimators=300, pca__n_components=20; total time=   2.3s
[CV] END ada__learning_rate=1e-05, ada__n_estimators=10, pca__n_components=9; total time=   0.0s
[CV] END ada__learning_rate=1e-05, ada__n_estimators=10, pca__n_components=9; total time=   0.0s
[CV] END ada__learning_rate=1e-05, ada__n_estimators=10, pca__n_components=9; total time=   0.1s
[CV] END ada__learning_rate=1e-05, ada__n_estimators=10, pca__n_components=9; total time=   0.1s
[CV] END ada__learning_rate=1e-05, ada__n_estimators=10, pca__n_components=9; total time=   0.1s
[CV] END ada__learning_rate=1e-05, ada__n_estimators=10, pca__n_components=9; total time=   0.1s
[CV] END ada__learning_rate=1e-05, ada__n_estimators=10, pca__n_components=9; total time=   0.1s
[CV] END ada__learning_rate=1e-05, ada__n_estimators=10, pca__n_components=9; total time=   0.1s
[CV] END ada__learning_rate=0.001, ada__n_estimators=50, pca__n_components=11; total ti

[CV] END pca__n_components=3, rf__criterion=absolute_error, rf__max_depth=25, rf__n_estimators=500; total time= 1.1min
[CV] END pca__n_components=9, rf__criterion=absolute_error, rf__max_depth=None, rf__n_estimators=100; total time=  30.6s
[CV] END criterion=squared_error, max_depth=5, n_estimators=500; total time=   4.1s
[CV] END criterion=friedman_mse, max_depth=None, n_estimators=200; total time=  17.7s
[CV] END criterion=friedman_mse, max_depth=None, n_estimators=200; total time=  53.0s
[CV] END criterion=friedman_mse, max_depth=None, n_estimators=50; total time=   4.9s
[CV] END criterion=friedman_mse, max_depth=None, n_estimators=50; total time=   9.3s
[CV] END criterion=friedman_mse, max_depth=25, n_estimators=150; total time=   9.8s
[CV] END criterion=friedman_mse, max_depth=25, n_estimators=150; total time=  37.5s
[CV] END criterion=absolute_error, max_depth=5, n_estimators=100; total time= 2.3min
[CV] END ada__learning_rate=0.001, ada__n_estimators=10, pca__n_components=12; to

[CV] END ada__learning_rate=1e-05, ada__n_estimators=300, pca__n_components=20; total time=   5.2s
[CV] END ada__learning_rate=1e-05, ada__n_estimators=300, pca__n_components=20; total time=   4.6s
[CV] END ..................learning_rate=1, n_estimators=300; total time=   6.2s
[CV] END .............learning_rate=0.0001, n_estimators=100; total time=   0.8s
[CV] END .............learning_rate=0.0001, n_estimators=100; total time=   1.6s
[CV] END .............learning_rate=0.0001, n_estimators=100; total time=   4.9s
[CV] END ..............learning_rate=1e-05, n_estimators=250; total time=  15.3s
[CV] END ..................learning_rate=1, n_estimators=100; total time=   2.9s
[CV] END ..............learning_rate=0.001, n_estimators=250; total time=   2.2s
[CV] END ..............learning_rate=0.001, n_estimators=250; total time=  10.5s
[CV] END ...............learning_rate=0.01, n_estimators=100; total time=   7.5s
[CV] END pca__n_components=13, xg__eta=0.6, xg__max_depth=6, xg__scale_po

[CV] END ada__learning_rate=1e-05, ada__n_estimators=300, pca__n_components=20; total time=   3.1s
[CV] END ada__learning_rate=0.001, ada__n_estimators=50, pca__n_components=11; total time=   0.2s
[CV] END ada__learning_rate=0.001, ada__n_estimators=50, pca__n_components=11; total time=   0.3s
[CV] END ada__learning_rate=0.001, ada__n_estimators=50, pca__n_components=11; total time=   0.7s
[CV] END ada__learning_rate=0.001, ada__n_estimators=50, pca__n_components=11; total time=   0.8s
[CV] END ada__learning_rate=1e-05, ada__n_estimators=200, pca__n_components=14; total time=   2.9s
[CV] END ada__learning_rate=1e-05, ada__n_estimators=200, pca__n_components=14; total time=   2.3s
[CV] END ..................learning_rate=1, n_estimators=300; total time=   4.0s
[CV] END ................learning_rate=0.1, n_estimators=150; total time=   7.6s
[CV] END ..............learning_rate=1e-05, n_estimators=250; total time=   6.3s
[CV] END ..............learning_rate=1e-06, n_estimators=250; total 